In [23]:
from bam_core.functions.analyze_fulfilled_requests import AnalyzeFulfilledRequests
from bam_core.lib.airtable import Airtable

In [25]:
airtable = Airtable()

In [4]:
grouped_records = AnalyzeFulfilledRequests().get_grouped_records()

[INFO] - 10:12:31 -> Fetching snapshots from Digital Ocean Space...


In [11]:
record_ids_to_change = set()

In [35]:
for record_id, records in grouped_records.items():
    last_record = {}
    for record in sorted(records, key=lambda x: x['Snapshot Date']):
        ss_requests = last_record.get('Social Services Requests?', [])
        eg_requests = record.get('Essential Goods Requests?', [])
        if any([req.startswith('Ropa') for req in eg_requests]): 
            if any([req.startswith('Historical: Asistencia de Ropa') for req in ss_requests]):
                record_ids_to_change.add(record_id)
        last_record = record

In [36]:
clothing_tag = 'Ropa / Clothing / 服装协助'
ss_tag = 'Historical: Asistencia de Ropa / Clothing Assistance'

In [ ]:
for record_id in list(record_ids_to_change):
    record = airtable.assistance_requests.get(record_id)
    eg_requests = record.get('fields', {}).get('Essential Goods Requests?', [])
    ss_requests = record.get('fields', {}).get('Social Services Requests?', [])
    if clothing_tag in eg_requests and ss_tag not in ss_requests:
        eg_requests.remove(clothing_tag)
        ss_requests.append(ss_tag)
        assert clothing_tag not in eg_requests
        assert ss_tag in ss_requests
        airtable.assistance_requests.update(record_id, {'Essential Goods Requests?': eg_requests, 'Social Services Requests?': ss_requests})